<a href="https://colab.research.google.com/github/sayanbanerjee32/TASI_ERAv2_S20/blob/main/bengali_bpe_tokenizer_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# assignment repo
!git clone https://github.com/sayanbanerjee32/TASI_ERAv2_S20.git

Cloning into 'TASI_ERAv2_S20'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 24 (delta 7), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (24/24), 19.78 KiB | 3.30 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [3]:
# copy the begali corpus files
!cp /content/drive/MyDrive/ERAv2_datasets/bengali_corpus/ben.txt .
!cp /content/drive/MyDrive/ERAv2_datasets/bengali_corpus/dev_bn_short.txt .

# copy utils file
!mv TASI_ERAv2_S20/utils.py .

In [4]:
# read the corpus files
with open('ben.txt', 'r') as _f:
    # raw_text = _f.readlines()
    raw_lines = [line.split('\t')[1] for line in _f]
    text1 = '\n'.join(raw_lines)

with open('dev_bn_short.txt', 'r') as _f:
    text2 = ''.join(_f.readlines())


text1[-1000:], text2[:1000]

('্য সবাই যা করতে চায় তা করতে চায়নি।\nএকা থাকার একটা অসুবিধে হোলো যে কথা বলার মতো কেউ থাকে না।\nআপনার সমস্ত অর্থ দিয়ে আপনি যা চান তা কিনতে সক্ষম হওয়া উচিত।\nবাইরে এতই গরম যে আমি পুরোদিন আমার শীততাপ নিয়ন্ত্রিত বাড়িতে থাকতে চাই।\nসকালে সূর্য পূর্ব দিকে ওঠে এবং সন্ধ্যায় পশ্চিমে অস্ত যায়।\nআমি শুনলাম যে তাঁরা পার্ক স্ট্রিটের একটা বাড়ির ভিতের মধ্যে থেকে একটা কঙ্কাল পেয়েছেন।\nটম যখন শুনলো যে মেরি আরে জন বিয়ে করেছে তখন তাকে দেখে বেশ অবাক মনে হয়েছিলো।\nআমার মনে হয় টম যে সোনা পেয়েছে সেটা তার কাছে রাখতে দেওয়া হবে এমন সম্ভাবনা খুব কম।\nটম মেরিকে বললো যে ও নিজেকে হত্যা করতে চলেছিলো, কিন্ত তা করার মতো সাহস ছিলো না।\nটমের সঙ্গে কাজ করা খুব বিরক্তিকর কারণ ও কখনই মেনে নেয় না যে ও ভুল করেছে।\nআমি ভেবেছিলাম এটা করা সহজ হবে, কিন্তু আমরা সারাদিন ধরে কাজ করেছি আর এখনো শেষ করে উঠতে পারিনি।\nআমি ভেবেছিলাম এটা করা সহজ হবে, কিন্তু আমরা সারাদিন ধরে কাজ করেছি আর এখনো শেষ করে উঠতে পারিনি।\nবছরের বারোটা মাস হলো জানুয়ারি, ফেব্রুয়ারি, মার্চ, এপ্রিল, মে, জুন জুলাই, আগস্ট, সেপ্টেম্বর, অক্টোবর, নভেম্ব

In [5]:
# merge all contents
ben_text = text1 + '\n' +text2

In [6]:
tokens = ben_text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience

## Bengali tokenizer with regex

In [7]:
import regex as re
from utils import get_stats, merge, encode, decode

In [8]:
vocab_size = 5001 # the desired final vocabulary size
num_merges = vocab_size - 256

In [9]:
# regex pattern for bengali text
regex_pat = re.compile(r""" ?\p{Bengali}+| ?[^\s\p{Bengali}]+|\s+(?!\S)|\s+""")

In [10]:
# split the text up into text chunks
text_chunks = re.findall(regex_pat, ben_text)

In [11]:
# input text preprocessing
ids = [list(ch.encode("utf-8")) for ch in text_chunks]

In [12]:
# iteratively merge the most common pairs to create new tokens
merges = {} # (int, int) -> int
vocab = {idx: bytes([idx]) for idx in range(256)} # idx -> bytes
for i in range(num_merges):
    # count the number of times every consecutive pair appears
    stats = {}
    for chunk_ids in ids:
        # passing in stats will update it in place, adding up counts
        get_stats(chunk_ids, stats)
    # find the pair with the highest count
    pair = max(stats, key=stats.get)
    if stats[pair] < 2: # there should be at least 2 occurances
        print("exitting - less than 2 occurances")
        break
    # mint a new token: assign it the next available id
    idx = 256 + i
    # replace all occurrences of pair in ids with idx
    ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]
    # save the merge
    merges[pair] = idx
    vocab[idx] = vocab[pair[0]] + vocab[pair[1]]


print(f"vocab size: {len(vocab)}")
print(f"Number of merges: {i}")
print(f"Number of occurences of last merged tokens: {stats[pair]}")

vocab size: 5001
Number of merges: 4744
Number of occurences of last merged tokens: 6


In [13]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 1344947
ids length: 121387
compression ratio: 11.08X


## Validate

In [14]:
## Test whether encoding and then decoding of training text results into same text
text2 = decode(encode(ben_text, regex_pat, merges),vocab)
print(text2 == ben_text)

True


In [15]:
valtext = """
বিশেষভাবে হত্যাকা-ের শিকার হচ্ছে শিশুরা।
অন্যদের কোনওরকমভাবে হয়রান করা হচ্ছে না।
বিক্ষোভকারীদের ওপর পুলিশ কাঁদানে গ্যাস ছোড়ে।
সেটা ফেরত পাওয়া খুব সোজা ব্যাপার নয়, অনিমেষ।
পরে হাসপাতাল থেকেও তাঁকে বের করে দেওয়া হয়।
নয়তো কার?
সম্প্রতি শারীরিক অসুস্থতার কারণে তাঁকে ভর্তি করা হয় সেখানকার হাসপাতালে।
ইতিমধ্যে ছবির শুটিংও শুরু হয়ে গেছে মুম্বইয়ের ধারাবি অঞ্চলে।
"""
valtext2 = decode(encode(valtext, regex_pat, merges),vocab)
print(valtext2 == valtext)

True


### Save the vocab, merges and the regex

In [16]:
import pickle

In [17]:
save_dict = {"vocab": vocab, "merges": merges, "regex_pat": regex_pat}
with open("bengali_tokenizer.pkl", "wb") as f:
    pickle.dump(save_dict, f)

In [18]:
import os
os.makedirs('to_upload', exist_ok=True)

In [19]:
!cp bengali_tokenizer.pkl to_upload/

In [20]:
from huggingface_hub import HfApi
from google.colab import userdata
userdata.get('HF_TOKEN')
api = HfApi()

In [21]:
api.upload_folder(
    folder_path="./to_upload",
    repo_id="sayanbanerjee32/bengali_tokenizer",
    repo_type="model",
)

bengali_tokenizer.pkl:   0%|          | 0.00/144k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sayanbanerjee32/bengali_tokenizer/commit/2e728b823084d5d55fed6fdc5b504dd99fc6940b', commit_message='Upload folder using huggingface_hub', commit_description='', oid='2e728b823084d5d55fed6fdc5b504dd99fc6940b', pr_url=None, pr_revision=None, pr_num=None)